In [ ]:
from GLM import datasim
from GLM import models
from GLM import glm
from GLM import utils as ut
from GLM import DesignMaker as dm
from PacTimeOrig.data import DataHandling as dh
from PacTimeOrig.data import DataProcessing as dp
import pandas as pd
import patsy
import numpy as np
import jax.numpy as jnp
import numpyro
import arviz as az

cfgparams={'area':'pMD'}

In [ ]:

#load datafile
datafile=dh.dataloader(sess=3,region=cfgparams['area'],suffix='Pac_'+cfgparams['area']+'.mat')

#Get session variables
sessvars=dh.getvars(datafile)

#Get position data
positions=dh.retrievepositions(datafile,rescale = [960.00,540.00])
if cfgparams['area']=='pMD':
    psth=dh.get_psth(datafile,win_shift=30)
else:
    psth=dh.get_psth(datafile,win_shift=75)

kinematics=dp.computederivatives(positions, vartodiff=['selfXpos','selfYpos','prey1Xpos','prey1Ypos','prey2Xpos','prey2Ypos'], dt=1.0/60.0)
# sessvars = dp.get_reaction_time(sessvars,kinematics)
sessvars = dh.get_matlab_wt_reaction_time(sessvars,session=3,subj='H')
#Select 2 prey trials
ogsessvars=sessvars
kinematics,sessvars =dh.subselect(kinematics,sessvars,trialtype='2')
psth,_= dh.subselect(psth,ogsessvars,trialtype='2')
#Drop columns
kinematics=dh.dropcols(kinematics, columns_to_drop=['predXpos','predYpos','prey1Xaccel','prey1Yaccel','prey2Xaccel','prey2Yaccel'])

#Get W_t vector
wtvec=dh.get_wt_vector(folder_path='/Users/user/PycharmProjects/PacManMain/data/WtNHP/H/NHP_H_SESSION_3/',selectype='average',transform=True)

#Cut data to correct length of wt
kinematics = dh.cut_to_rt(kinematics, sessvars)
# psth = [pd.DataFrame(x) for x in psth]
psth = dh.cut_to_rt(psth, sessvars)
kinematics = dh.get_time_vector(kinematics)
kinematics = dp.compute_distance(kinematics,trialtype=2)
#compute relative normalized speed
kinematics = dp.compute_relspeed(kinematics,trialtype=2)
kinematics = dp.compute_selfspeed(kinematics)

# Now merge wt:
wt = wtvec['wt']
wt=[pd.DataFrame(x) for x in wt]
wt=[df.rename(columns={0: 'wt'}) for df in wt]
kinematics = [df.merge(wt[i], left_index=True, right_index=True) for i, df in enumerate(kinematics)]


#For each kinematics frame, add relative reward value
Xdsgn = kinematics
for trial in range(len(Xdsgn)):
    Xdsgn[trial]['val1'] = np.repeat(sessvars.iloc[trial].NPCvalA,len(kinematics[trial]))
    Xdsgn[trial]['val2'] = np.repeat(sessvars.iloc[trial].NPCvalB,len(kinematics[trial]))
    
#Switch reward positions so highest value is always in prey 1 slot
Xdsgn=dh.rewardalign(Xdsgn)
Xdsgn = [df[sorted(df.columns)] for df in Xdsgn]

#Compute relative value
Xdsgn = [df.assign(relvalue=df['val1']-df['val2']).round(2) for df in Xdsgn]


Make design matrices and partial distance from wt

In [ ]:
Xin = pd.concat(Xdsgn, axis=0, ignore_index=True)
Xin['relvalue']=pd.Categorical(Xin['relvalue']).codes+1

Yin = pd.concat(psth, axis=0, ignore_index=True)
Xin['wt']+=0.1
Xin['wt']-=0.5

# Step 1: Perform linear regression to get slope and intercept
slope, intercept = np.polyfit(Xin['reldist'], Xin['wt'], 1)

# Step 2: Predict B using the linear model
B_pred = slope * Xin['reldist'] + intercept

# Step 3: Compute residuals
residuals = Xin['wt'] - B_pred

Xin['wt']=residuals

Setup regressors

In [1]:
################ DO AS CONTINUOUS MODEL OVER REWARD
nbases=8

Xin['relvalue']=(Xin['relvalue']-3)

basis_x_list, S_list, tensor_basis, tensor_S, beta_x_names = dm.pac_cont_dsgn(Xin, params={'basismain':'bs','nbases':nbases,'basistypeval':'linear','nbasis':5})



y=Yin[17]

mod2fit=glm.PoissonGLMbayes()

mod2fit.add_data(y=jnp.array(y))
#Learn smoothness from data
mod2fit.define_model(model='prs_double_penalty',basis_x_list=basis_x_list, S_list=S_list,tensor_basis_list=None,S_tensor_list=None)
# mod2fit.define_model(model='prs_double_penalty',basis_x_list=basis_x_list, S_list=S_list,tensor_basis_list=None,S_tensor_list=None)

params={'fittype': 'vi', 'guide':'normal','visteps':20000,'optimtype':'scheduled'}
mod2fit.fit(params=params,beta_x_names=beta_x_names,fit_intercept=True,cauchy=2)
mod2fit.sample_posterior(5000).summarize_posterior(95).coeff_relevance()

#Fit a noise model
mod2fit.fit(baselinemodel=True)
mod2fit.sample_posterior(5000,baselinemodel=True)

mod2fit.model_metrics(getbaselinemetric=True)



# FIT WITH INTERACTIONS
mod2fitinter=glm.PoissonGLMbayes()

mod2fitinter.add_data(y=jnp.array(y))
#Learn smoothness from data
mod2fitinter.define_model(model='prs_double_penalty',basis_x_list=basis_x_list, S_list=S_list,tensor_basis_list=[tensor_basis[0]],S_tensor_list=[tensor_S[0]])


params={'fittype': 'vi', 'guide':'normal','visteps':20000,'optimtype':'scheduled'}
mod2fitinter.fit(params=params,beta_x_names=beta_x_names,fit_intercept=True,cauchy=2)
mod2fitinter.sample_posterior(5000).summarize_posterior(99).coeff_relevance()

#Fit a noise model
mod2fitinter.fit(baselinemodel=True)
mod2fitinter.sample_posterior(5000,baselinemodel=True)

mod2fitinter.model_metrics(getbaselinemetric=True)






comparison = az.compare({'model_inter': mod2fitinter.model_idata, 'model_base': mod2fit.model_idata}, ic="waic")





###### THIS IS PLOTTING CODE THAT NEEDS TO BE PUT IN THE CLASS
#Full prediction space
# Define ranges for x2 and x3
x2_range = jnp.linspace(jnp.min(Xin['reldist'].values), jnp.max(Xin['reldist'].values), 100)
x3_range = jnp.linspace(jnp.min(Xin['wt'].values), jnp.max(Xin['wt'].values), 100)

# Create a meshgrid to evaluate the interaction
X2, X3 = jnp.meshgrid(x2_range, x3_range)

# Flatten the meshgrid for use in patsy
x2_flat = X2.ravel()
x3_flat = X3.ravel()

# Generate main effect basis matrices for x2 and x3
basis_x2 = patsy.dmatrix("bs(x2, degree=3, df=nbases) - 1", {"x2": x2_flat, "nbases": nbases}, return_type="dataframe")
basis_x3 = patsy.dmatrix("bs(x3, degree=3, df=nbases) - 1", {"x3": x3_flat, "nbases": nbases}, return_type="dataframe")

# Generate the tensor product basis matrix
tensor_basis = patsy.dmatrix("te(bs(x2, degree=3, df=nbases), bs(x3, degree=3, df=nbases)) - 1",
                             {"x2": x2_flat, "x3": x3_flat, "nbases": nbases},
                             return_type="dataframe")


samples_x2 = np.array(mod2fitinter.posterior_samples['beta_beta_x2'])
adjbetas_x2 = samples_x2 * mod2fit.coef_keep['beta_beta_x2']
mean_betas_x2 = adjbetas_x2.mean(axis=0)

samples_x3 = np.array(mod2fitinter.posterior_samples['beta_beta_x3'])
adjbetas_x3 = samples_x3 * mod2fit.coef_keep['beta_beta_x3']
mean_betas_x3 = adjbetas_x3.mean(axis=0)

tensor_samples = np.array(mod2fitinter.posterior_samples['beta_tensor_0'])
adj_tensor_betas = tensor_samples * mod2fitinter.coef_keep['beta_tensor_0']
mean_tensor_betas = adj_tensor_betas.mean(axis=0)

intercept = mod2fitinter.posterior_means['intercept']
pred_main_x2 = np.array(basis_x2) @ mean_betas_x2.T
pred_main_x3 = np.array(basis_x3) @ mean_betas_x3.T
pred_tensor = np.array(tensor_basis) @ mean_tensor_betas.T


# Combine all effects and add intercept
predictions = np.exp((pred_main_x2 + pred_main_x3 + pred_tensor + intercept).reshape(100, 100))


from matplotlib import cm

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# Plot the surface.
surf = ax.plot_surface(X2, X3, predictions, cmap=cm.coolwarm,
                       linewidth=1, antialiased=False)



predictions = np.exp(pred_main_x3.reshape(100,100)[:,:].mean(axis=0) +intercept)



SyntaxError: unterminated string literal (detected at line 55) (469483415.py, line 55)

What to save: coef summary, predictions and hdpi, comparisons